In [1]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Mounted at /content/drive


In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 5.4 MB/s 
     |████████████████████████████████| 3.5 MB 4.3 MB/s 
     |████████████████████████████████| 2.4 MB 1.3 MB/s 


In [3]:
train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
#from torch.utils.data import DataLoader
from torch_geometric.utils import degree
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx





%cd /content/drive/MyDrive/ADNI2/Controls
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp)  
      tmp[tmp<=0.4]=0
      graphlist.append([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])

%cd /content/drive/MyDrive/ADNI2/Patients
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp) 
      tmp[tmp<=0.4]=0
      graphlist.append([tmp.type(torch.FloatTensor),torch.zeros(1).type(torch.long)])

/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Controls
/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Patients


In [4]:
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [5]:
class E2EBlock(torch.nn.Module):
    def __init__(self, in_planes, planes,bias=True):
        super(E2EBlock, self).__init__()
        self.d = 90
        self.cnn1 = torch.nn.Conv2d(in_planes,planes,(1,self.d),bias=bias)
        self.cnn2 = torch.nn.Conv2d(in_planes,planes,(self.d,1),bias=bias)       
    def forward(self, x):
        a = self.cnn1(x)
        b = self.cnn2(x)
        return torch.cat([a]*self.d,3)+torch.cat([b]*self.d,2)

class Brainnet(torch.nn.Module):
    def __init__(self):
        super(Brainnet, self).__init__()
        self.d = 90
        self.e2econv1 = E2EBlock(1,32)   ######### Changed input channel from 1 to 3
        self.e2econv2 = E2EBlock(32,64)
        self.E2N = torch.nn.Conv2d(64,1,(1,self.d))
        self.N2G = torch.nn.Conv2d(1,256,(self.d,1))
        self.dense1 = torch.nn.Linear(256,128)
        self.dense2 = torch.nn.Linear(128,30)
        self.aux = torch.nn.Linear(30,1)
      
    def forward(self, x):
        out = F.leaky_relu(self.e2econv1(x),negative_slope=0.33)
        out = F.leaky_relu(self.e2econv2(out),negative_slope=0.33) 
        out = F.leaky_relu(self.E2N(out),negative_slope=0.33)
        out = F.leaky_relu(self.N2G(out),negative_slope=0.33)
        out = out.view(out.size(0), -1)
        out = F.leaky_relu(self.dense1(out),negative_slope=0.33)
        out = F.leaky_relu(self.dense2(out),negative_slope=0.33)
        classes = self.aux(out) 
        return classes

In [6]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Brainnet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data, label in loader:  # Iterate in batches over the training/test dataset.
         label_classes = label.type(torch.float).cuda() 
         cur_batch_size = torch.tensor(len(data)).to(device)
         data = data.view(cur_batch_size, -1).to(device)
         data = data.view(cur_batch_size, 1, 90, 90)
         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         #print(label_classes.size(), "label")
         #print(pred.size(), "pred")
         label_classes = label_classes.view(-1)
         correct += int((pred.view(-1) == label_classes).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 30):
    model.train()

    for data, label in train_loader:  # Iterate in batches over the training dataset.
         label_classes = label.type(torch.float).cuda() 
         cur_batch_size = torch.tensor(len(data)).to(device)
         data = data.view(cur_batch_size, -1).to(device)
         data = data.view(cur_batch_size, 1, 90, 90)
         out = model(data.cuda())
         out = out.view(-1)
         label = label.view(-1).cuda()
         #print(label.size(), "label")
         loss = criterion(out, label.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4933, Test Acc: 0.5147, Loss: 0.6924
Epoch: 002, Train Acc: 0.9133, Test Acc: 0.9706, Loss: 0.5660
Epoch: 003, Train Acc: 0.8733, Test Acc: 0.9412, Loss: 1.1957
Epoch: 004, Train Acc: 0.9400, Test Acc: 0.8676, Loss: 0.3784
Epoch: 005, Train Acc: 0.9667, Test Acc: 1.0000, Loss: 0.1060
Epoch: 006, Train Acc: 0.9800, Test Acc: 0.8971, Loss: 0.1669
Epoch: 007, Train Acc: 0.9867, Test Acc: 0.9853, Loss: 0.2536
Epoch: 008, Train Acc: 0.9933, Test Acc: 0.9118, Loss: 0.1066
Epoch: 009, Train Acc: 1.0000, Test Acc: 0.9412, Loss: 0.0145
Epoch: 010, Train Acc: 1.0000, Test Acc: 0.9706, Loss: 0.0008
Epoch: 011, Train Acc: 1.0000, Test Acc: 0.9706, Loss: 0.0010
Epoch: 012, Train Acc: 1.0000, Test Acc: 0.9265, Loss: 0.0017
Epoch: 013, Train Acc: 1.0000, Test Acc: 0.9265, Loss: 0.0018
Epoch: 014, Train Acc: 1.0000, Test Acc: 0.9265, Loss: 0.0003
Epoch: 015, Train Acc: 1.0000, Test Acc: 0.9559, Loss: 0.0003
Epoch: 016, Train Acc: 1.0000, Test Acc: 0.9853, Loss: 0.0000
Epoch: 0